# Creazione modello text analysis
Ho visto che il decision tree è il modello che mi offre migliori performance, soprattutto con TfIdf e tokenizzazione nltk/regex con stemming PorterStemmer().


In [6]:
from my_tokenizer import MyTokenizer
#from classification import Classification
#from model import Model
import pandas as pd
import util_strings as utils

È necessaria la normalizzazione delle feature, dato che nel testo abbiamo i nomi delle squadre ed i soprannomi, vado a sostituire tutte le parole relative alla squadra di casa con home team e stessa cosa per la away team.
Questo viene fatto tramite il metodo feature_normalization()

In [7]:
mt = MyTokenizer(pd.read_csv(utils.completed_dataset, index_col=0))
mt.feature_normalization()
#mt.dataset

In [8]:
mt.clean_text()
X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets()
#mt.save_vectorizer(utils.vectorizer)
#mt.dataset

## Classificazione del modello
Migliori risultati con Decision Tree.
Viene salvato il modello

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model = DecisionTreeClassifier()

In [10]:
import sklearn.metrics as mtr
E = []

accuracy = 0
while(accuracy < 0.834):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    analysis = pd.DataFrame()
            
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    if report['accuracy'] > accuracy: 
        accuracy = report['accuracy']
        print(accuracy)

0.7484662576687117
0.754601226993865
0.7607361963190185


KeyboardInterrupt: 

In [ ]:
analysis = {
    'Model': 'Model', 'Accuracy': report['accuracy'],
    'Avg Precision (macro)': report['macro avg']['precision'],
    'Avg Recall (macro)': report['macro avg']['recall'],
    'Avg F1-score (macro)': report['macro avg']['f1-score'],
    'Avg Precision (weighted)': report['weighted avg']['precision'],
    'Avg Recall (weighted)': report['weighted avg']['recall'],
    'Avg F1-score (weighted)': report['weighted avg']['f1-score']
}
analysis

{'Model': 'Model',
 'Accuracy': 0.8220858895705522,
 'Avg Precision (macro)': 0.8271607461051805,
 'Avg Recall (macro)': 0.7946918935492889,
 'Avg F1-score (macro)': 0.8073618724918153,
 'Avg Precision (weighted)': 0.827820404993625,
 'Avg Recall (weighted)': 0.8220858895705522,
 'Avg F1-score (weighted)': 0.8212416267302488}

In [ ]:
import pickle

In [ ]:
#with open(utils.text_analysis, 'wb') as file:
#    pickle.dump(model, file)